In [12]:
import csv
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [13]:
from feature import Feature
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatool.preprocess import  Preprocessor
from datatool.maneger import DataManager

In [14]:
path = "../corpus/data_augment.csv"

In [15]:
data = []
with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)

In [35]:
import spacy
nlp = spacy.load('ja_ginza')

In [17]:
def make_classes_dict(data):
    classes_dict = dict()
    for row in data:
        c = row[1]
        if c not in classes_dict:
            classes_dict[c] = len(classes_dict)
    return classes_dict

In [18]:
def class_filter(data, remain_classes:list):
    new_data = []
    for row in data:
        c = row[1]
        if c in remain_classes:
            new_data.append(row)
    return new_data

In [19]:
classes_dict_ = make_classes_dict(data)

In [20]:
remain_classes = "how what when where who why YN plain".split()
data_n = class_filter(data, remain_classes)
classes_dict = dict(zip(remain_classes, list(range(len(remain_classes)))))

In [21]:
def extract_X_y(data, classes_dict):
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(classes_dict[d[1]])
    return X, y

In [22]:
import random
def extract_X_y_limit(data, classes_dict:dict, limit=400):
    data_ = random.sample(data, len(data))
    X = []
    y = []
    each_len = dict(zip(classes_dict.keys(), [0]*len(classes_dict)))
    for d in data_:
        if each_len[d[1]] <= limit:
            X.append(d[0])
            y.append(classes_dict[d[1]])
            each_len[d[1]] += 1
    return X, y

In [23]:
# X, y = extract_X_y(data_n, classes_dict)

In [24]:
X, y = extract_X_y_limit(data_n, classes_dict)

In [25]:
len(X)

3208

In [26]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [27]:
F = Feature()
F.set_preprocessor(Preprocessor())
F.make_features(X_train_str)

X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

In [28]:
F_path = "../X_y_data/"
F_name = "typeClassify_F2.pickle"
# import dill
import pickle
# featureM = DataManager(F_path, format_="dill")
with open(F_path+F_name, "wb") as f:
    pickle.dump(F, f)

In [29]:
lr = LogisticRegression(solver='sag', max_iter=100)
lr.fit(X_train, y_train)

/home/yamada/.local/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(solver='sag')

In [30]:
y_pred = lr.predict(X_test)

In [31]:
print(classes_dict)
dis = 30
for y_p, x_s in zip(y_pred[:dis], X_test_str[:dis]):
    print("{0} : {1}".format(y_p, x_s))

{'how': 0, 'what': 1, 'when': 2, 'where': 3, 'who': 4, 'why': 5, 'YN': 6, 'plain': 7}
7 : 近ごろヨーロッパで盛んになっているという「ナチュラルムーブメント」について耳にしますが、これは何ですか。
5 : なぜここに？
4 : 配役のためのオーディションを受けたいのはだれ？
6 : 実際、日本で9月1日は防災の日になっています。
0 : 1回につきいくらかかるのですか。
5 : 美容師さんにもっと明るくするようにお願いしてみたら？そうすれば、あなたの髪の色は私のと、ちょうど同じになるでしょう！
1 : 何だって？
0 : スウェットワーキングの流行は、ここアメリカではどのように始まったのですか。
5 : どうしてわたしに電話をしているの？
2 : 締め切りはいつですか？
6 : 私はお店に向かうところです。何か買ってきましょうか？
6 : これが、メッセージの最初の文字なの？
3 : それじゃあ、どこへ行きたいのさ？
4 : あなたは、だれについて話しているの？
2 : 智美の出る競技は、いつ始まるかな？
3 : どこへ行くべきだろう？
5 : わたしの言ったとおり、翔さんは気に入ってくれたでしょう。
6 : これはと思うような切り出し文句を使っていますか。
6 : 今夜、夕食でもどう？
3 : なぜそこへ行きたいの？
0 : この状況をどう判断しますか？
6 : 「宝」って言った？
5 : あなたたちは、どうしてスナップドラゴンの手下になっているの？
6 : お孫さんは何人になられたのですか。
5 : あなたたちは、どうしてスナップドラゴンの手下になっているの？
7 : 家族や友だちといっしょに過ごしている。
0 : わたしたちの計画をどう思いますか？
7 : はせ参じました、閣下。
4 : それは、だれのかしら？
4 : あの男性はだれじゃ？


In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[ 83  10   0   1   0   6  10   3]
 [ 15  81   1   2   1   3  10   2]
 [  0   0 128   0   2   0   2   1]
 [  0   2   0 114   0   1   0   0]
 [  0   3   0   4 124   1   3   1]
 [  0   4   0   1   1  96  14   5]
 [  7   3   2   0   6   8  83  10]
 [  6   5   1   0   1   8  23  65]]
accuracy =  0.8037383177570093


In [33]:
model_path = "../models/"
model_name = "typeClassify_M2.pickle"
modelM = DataManager(model_path)
print(model_name)

typeClassify_M2.pickle


In [34]:
modelM.save_data(model_name, lr)

success save : ../models/typeClassify_M2.pickle
